In [ ]:
#!pip install anvil-uplink

In [52]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

#for pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

import anvil.server

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/raw-data/raw_data.xlsx


In [53]:
data = pd.read_excel("../input/raw-data/raw_data.xlsx")

In [54]:
data

,Disease,Count of Disease Occurrence,Symptom
0,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0008031_pain chest
1,NaN,NaN,UMLS:C0392680_shortness of breath
2,NaN,NaN,UMLS:C0012833_dizziness
3,NaN,NaN,UMLS:C0004093_asthenia
4,NaN,NaN,UMLS:C0085639_fall
...,...,...,...
1861,NaN,NaN,UMLS:C0425251_bedridden^UMLS:C0741453_bedridden
1862,NaN,NaN,UMLS:C0242453_prostatism
1863,UMLS:C0011127_decubitus ulcer,42.0,UMLS:C0232257_systolic murmur
1864,NaN,NaN,UMLS:C0871754_frail


In [55]:
data = data.fillna(method='ffill')

In [56]:
data['Disease'] = data['Disease'].str.split("^",expand = True)[0]
data['Symptom'] = data['Symptom'].str.split("^",expand = True)[0]

In [57]:
data['Disease'] = data['Disease'].str.split("_",expand = True)[1]
data['Symptom'] = data['Symptom'].str.split("_",expand = True)[1]

In [58]:
data_new = data.copy()

In [59]:
def renew_data(data, col1, col2):
    i = 1
    while i<data.shape[0]:
        if data[col1].iloc[i]==data[col1].iloc[i-1]:
            data[col2].iloc[i] = data[col2].iloc[i-1]+" "+data[col2].iloc[i]
        i = i +1
    return data

In [60]:
renew_data(data_new,'Disease','Symptom')

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Disease,Count of Disease Occurrence,Symptom
0,hypertensive disease,3363.0,pain chest
1,hypertensive disease,3363.0,pain chest shortness of breath
2,hypertensive disease,3363.0,pain chest shortness of breath dizziness
3,hypertensive disease,3363.0,pain chest shortness of breath dizziness asthenia
4,hypertensive disease,3363.0,pain chest shortness of breath dizziness asthe...
...,...,...,...
1861,affect labile,45.0,extreme exhaustion sleeplessness enuresis pati...
1862,affect labile,45.0,extreme exhaustion sleeplessness enuresis pati...
1863,decubitus ulcer,42.0,systolic murmur
1864,decubitus ulcer,42.0,systolic murmur frail


In [61]:
data_new.groupby('Disease').describe()

Count of Disease Occurrence              \
                                                          count   mean  std   
Disease                                                                       
Alzheimer's disease                                        17.0  101.0  0.0   
Pneumocystis carinii pneumonia                             18.0  113.0  0.0   
accident cerebrovascular                                    8.0  885.0  0.0   
acquired immuno-deficiency syndrome                        14.0  350.0  0.0   
adenocarcinoma                                             12.0  166.0  0.0   
...                                                         ...    ...  ...   
tonic-clonic epilepsy                                      17.0   92.0  0.0   
transient ischemic attack                                  17.0  168.0  0.0   
tricuspid valve insufficiency                              10.0  101.0  0.0   
ulcer peptic                                               12.0  143.0  0.0   
upper respiratory infection                                28.0  135.0  0.0   

                                                                        
                                       min    25%    50%    75%    max  
Disease                                                                 
Alzheimer's disease                  101.0  101.0  101.0  101.0  101.0  
Pneumocystis carinii pneumonia       113.0  113.0  113.0  113.0  113.0  
accident cerebrovascular             885.0  885.0  885.0  885.0  885.0  
acquired immuno-deficiency syndrome  350.0  350.0  350.0  350.0  350.0  
adenocarcinoma                       166.0  166.0  166.0  166.0  166.0  
...                                    ...    ...    ...    ...    ...  
tonic-clonic epilepsy                 92.0   92.0   92.0   92.0   92.0  
transient ischemic attack            168.0  168.0  168.0  168.0  168.0  
tricuspid valve insufficiency        101.0  101.0  101.0  101.0  101.0  
ulcer peptic                         143.0  143.0  143.0  143.0  143.0  
upper respiratory infection          135.0  135.0  135.0  135.0  135.0  

[134 rows x 8 columns]

In [62]:
cv = CountVectorizer()

In [63]:
X = data_new['Symptom']
Y = data_new['Disease']

In [64]:
X = cv.fit_transform(X)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.3,random_state=101)

In [66]:
#nb = MultinomialNB()

In [ ]:
#nb.fit(X_train,y_train)

In [67]:
rfc = RandomForestClassifier()

In [69]:
rfc.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [77]:
predictions = rfc.predict(X_train)
print(confusion_matrix(y_train,predictions))
print('\n')
print(classification_report(y_train,predictions))

[[11  0  0 ...  0  0  0]
 [ 0 12  0 ...  0  0  0]
 [ 0  0  5 ...  0  0  0]
 ...
 [ 0  0  0 ...  5  0  0]
 [ 0  0  0 ...  0  9  0]
 [ 0  0  0 ...  0  0 23]]


                                      precision    recall  f1-score   support

                 Alzheimer's disease       1.00      1.00      1.00        11
      Pneumocystis carinii pneumonia       1.00      1.00      1.00        12
            accident cerebrovascular       0.83      1.00      0.91         5
 acquired immuno-deficiency syndrome       1.00      0.92      0.96        13
                      adenocarcinoma       1.00      0.67      0.80         6
                            adhesion       1.00      1.00      1.00        12
                       affect labile       1.00      1.00      1.00         7
                              anemia       1.00      1.00      1.00        13
                       anxiety state       1.00      1.00      1.00        21
                             aphasia       1.00      1.00    

In [78]:
test_predictions = rfc.predict(X_test)
print(confusion_matrix(y_test,test_predictions))
print('\n')
print(classification_report(y_test,test_predictions))

[[6 0 0 ... 0 0 0]
 [0 6 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 4 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 5]]


                                      precision    recall  f1-score   support

                 Alzheimer's disease       1.00      1.00      1.00         6
      Pneumocystis carinii pneumonia       1.00      1.00      1.00         6
            accident cerebrovascular       0.67      0.67      0.67         3
 acquired immuno-deficiency syndrome       1.00      1.00      1.00         1
                      adenocarcinoma       1.00      1.00      1.00         6
                            adhesion       1.00      1.00      1.00         1
                       affect labile       1.00      1.00      1.00         1
                              anemia       1.00      0.86      0.92         7
                       anxiety state       1.00      1.00      1.00         3
                             aphasia       1.00      1.00      1.00         2
                   

In [73]:
z = []
t = input().lower()
z.append(t)
z = pd.DataFrame(z, columns=['ABC'])
z = z['ABC']
z = cv.transform(z)
z_pred = rfc.predict(z)
z_pred[0]

swelling pain


'cellulitis'

In [74]:
anvil.server.connect('DLBMGK5KNJUILT25GCZD7AQK-E3R23QQLKE2LCHQY')
@anvil.server.callable
def classify_disease(sym):
    z = []
    z.append(sym)
    z = pd.DataFrame(z, columns=['ABC'])
    z = z['ABC']
    z = cv.transform(z)
    z_pred = rfc.predict(z)
    return z_pred[0]

# **PIPELINE**

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [ ]:
X_pipe = data_new['Symptom']
Y_pipe = data_new['Disease']

In [ ]:
pipeline.fit(X_pipe,Y_pipe)

In [ ]:
pred = pipeline.predict(data_new['Symptom'])

In [ ]:
print(confusion_matrix(data_new['Disease'],pred))
print(classification_report(data_new['Disease'],pred))